In [1]:

from __future__ import print_function

import sys

sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm


from datetime import datetime
from pytz import timezone
import itertools
import copy

import SimpleITK as sitk
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from config.config import config
from utils.experiment import Experiment, ExperimentHandler
from utils.batch_handlers import TwoDimBatchHandler
from utils.test_handler import ACDC2017TestHandler, test_ensemble
from utils.test_results import TestResults
from plotting.learning_curves import loss_plot
from plotting.uncertainty_plots import analyze_slices
from plotting.slice_outliers_plots import plot_outlier_slice_hists
from utils.generate_uncertainty_maps import ImageOutliers

from models.dilated_cnn import BaseDilated2DCNN

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"
LOG_DIR = os.path.join(ROOT_DIR, "logs")
CYCLE_OFFSET = 100000
# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
np.random.seed(SEED)

# IMPORTANT - THE FOLD WE'RE USING FOR TESTING. NOTE: Training and testing fold should be the same
# we raise an error in the test_ensemble method if this is not the case!
TEST_FOLD = 0

# baseline: 
# exp_base = {2: "20180328_10_53_16_dcnnv1_150000E_lr2e02",
#            0: "20180330_09_56_39_dcnnv1_150000E_lr2e02"}
# exp_base_model = os.path.join(LOG_DIR, exp_base[TEST_FOLD])

# MC dropout models
# Fold 2: 
# Fold 0: 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
exp_mc005 = {2: "",
             0: "20180413_16_36_58_dcnn_mcv1_150000E_lr2e02"}
exp_mc005_path = os.path.join(LOG_DIR, exp_mc005[TEST_FOLD])

# MC dropout models
# Fold 2: 
# Fold 0: 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
exp_mc01 = {2: "",
            0: "20180416_08_25_43_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_path = os.path.join(LOG_DIR, exp_mc01[TEST_FOLD])


# This model was retrained on checkpoint 140000 of the baseline MC model 20180413_16_36_58_dcnn_mcv1_150000E_lr2e02
# @epoch0-> generate uMaps, trained for 20,000 epochs where uMaps are re-generated each 10,000 iterations
# IMPORTANT: the model was ONLY trained on outlier slices for 20,000 epochs (after pre-training of 140,000)
# Fold 2: 
# Fold 0: 20180413_17_03_22_dcnn_mcv1_150000E_lr2e02
exp_mc005_expert1 = {2: "",
                     0: "20180417_17_43_20_dcnn_mcv1_20000E_lr2e02"}
exp_mc005_expert1_path = os.path.join(LOG_DIR, exp_mc005_expert1[TEST_FOLD])

# Fold 2: 
# Fold 0: 20180418_15_02_05_dcnn_mcv1_150000E_lr2e02
exp_mc01_brier = {2: "",
                    0: "20180418_15_02_05_dcnn_mcv1_150000E_lr2e02"}
exp_mc01_brier_path = os.path.join(LOG_DIR, exp_mc01_brier[TEST_FOLD])

In [3]:
exper_mc005 = ExperimentHandler.load_experiment(exp_mc005_path)
exper_hdl_mc005 = ExperimentHandler(exper_mc005, use_logfile=False)
exper_hdl_mc005.set_root_dir(ROOT_DIR)
exper_hdl_mc005.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc005.exper.run_args.fold_ids[0],
    exper_hdl_mc005.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005.exper.model_name))

exper_mc01 = ExperimentHandler.load_experiment(exp_mc01_path)
exper_hdl_mc01 = ExperimentHandler(exper_mc01, use_logfile=False)
exper_hdl_mc01.set_root_dir(ROOT_DIR)
exper_hdl_mc01.set_model_name("MC-dropout fold-{} (p={})".format(exper_hdl_mc01.exper.run_args.fold_ids[0],
    exper_hdl_mc01.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01.exper.model_name))

exper_mc005_expert1 = ExperimentHandler.load_experiment(exp_mc005_expert1_path)
exper_hdl_mc005_expert1 = ExperimentHandler(exper_mc005_expert1, use_logfile=False)
exper_hdl_mc005_expert1.set_root_dir(ROOT_DIR)
exper_hdl_mc005_expert1.set_model_name("MC-dropout-expert1 fold-{} (p={})".format(
                                     exper_hdl_mc005_expert1.exper.run_args.chkpnt_freq,
                                     exper_hdl_mc005_expert1.exper.run_args.fold_ids[0],
                                     exper_hdl_mc005_expert1.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc005_expert1.exper.model_name))

exper_mc01_brier = ExperimentHandler.load_experiment(exp_mc01_brier_path)
exper_hdl_mc01_brier = ExperimentHandler(exper_mc01_brier, use_logfile=False)
exper_hdl_mc01_brier.set_root_dir(ROOT_DIR)
exper_hdl_mc01_brier.set_model_name("MC-dropout brier fold-{} (p={})".format(
                                exper_hdl_mc01_brier.exper.run_args.fold_ids[0],
                                exper_hdl_mc01_brier.exper.run_args.drop_prob))
print("Model name {}".format(exper_hdl_mc01_brier.exper.model_name))

try:
    del test_set
except:
    pass

test_set = ACDC2017TestHandler(exper_config=exper_hdl_mc005.exper.config, 
                               search_mask=config.dflt_image_name + ".mhd", fold_ids=[TEST_FOLD],
                               debug=False, batch_size=25, use_cuda=True, load_train=False, load_val=True,
                               use_iso_path=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180413_16_36_58_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-0 (p=0.05)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180416_08_25_43_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout fold-0 (p=0.1)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180417_17_43_20_dcnn_mcv1_20000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout-expert1 fold-10000 (p=0)
Load experiment from /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/stats/exper_stats.dll
Model name MC-dropout brier fold-0 (p=0.1)
INFO - Testhandler - >>> Search in val-dir for /home/jorg/repository/dcnn_acdc/data/Folds/fold0/validate/images_iso/*patient*.mhd <<<
INFO - File list contains 50 files, hence 25 patients


100%|██████████| 25/25 [00:01<00:00, 16.48it/s]

INFO - Successfully loaded 25 ED/ES patient pairs


In [4]:
mc_samples = 10
sample_weights = True
store_details = True
image_range = [20,21,22,23,24]
if not sample_weights:
    generate_stats = False
else:
    generate_stats = True
save_results = False
checkpoints = [150000]  #  [150000]
use_uncertainty = False

exper_handler = exper_hdl_mc01_brier

exper_handler.logger = None   # exper_hdl_base
exper_handler.exper.outliers_per_epoch = None

test_ensemble(test_set, exper_handler, mc_samples=mc_samples, sample_weights=sample_weights,  
              use_uncertainty=use_uncertainty, referral_threshold=0., image_range=image_range, 
              checkpoints=checkpoints,
              verbose=False, reset_results=True, store_details=store_details, 
              generate_stats=generate_stats, save_results=save_results, do_filter=True)


    
print("Ready")

Loading existing model loaded from checkpoint dir /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/checkpoints
Evaluating model MC-dropout brier fold-0 (p=0.1) - runID 1
INFO - Running test on 5 images with model from checkpoint 150000
Image 21-patient096 - Test accuracy: test loss -2.210	  dice(RV/Myo/LV): ES 0.88/0.88/0.90 --- ED 0.94/0.87/0.97
Image 21 - Test accuracy: test loss -2.210	 Hausdorff(RV/Myo/LV): ES 12.52/12.52/19.55 --- ED 10.26/24.44/5.19
Image 22-patient097 - Test accuracy: test loss -2.810	  dice(RV/Myo/LV): ES 0.85/0.90/0.94 --- ED 0.94/0.87/0.97
Image 22 - Test accuracy: test loss -2.810	 Hausdorff(RV/Myo/LV): ES 13.06/9.39/5.60 --- ED 9.80/5.60/5.77
Image 23-patient098 - Test accuracy: test loss -2.138	  dice(RV/Myo/LV): ES 0.91/0.89/0.94 --- ED 0.92/0.86/0.92
Image 23 - Test accuracy: test loss -2.138	 Hausdorff(RV/Myo/LV): ES 16.09/26.04/5.05 --- ED 9.90/12.29/23.84
Image 24-patient099 - Test accuracy: test loss -1.199	  dice(RV/My

In [89]:
mc_probs = exper_handler.test_results.mc_pred_probs[0]
std_stats = exper_handler.test_results.stddev_maps[0]
print(std_stats.shape)
sliceid = 7
c=1  # ES-RV
# get probs from slice 8 [#samples, classes, width, height, #slices]
mc_probs_slice = mc_probs[:, 1, :, :, 7]
# get stddev-maps [classes, width, height, #slices]
std_stats_slice = std_stats[1, :, :, 7]
print(std_stats_slice.shape)
print(mc_probs_slice.shape)

(8, 223, 264, 8)
(223, 264)
(10, 223, 264)


In [195]:
exper_handler = exper_hdl_mc01_brier
t_path = "/home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02//stats/test_results_25imgs_mc10_fold0_ep150000.dll"
test_results = TestResults.load_results(t_path)
print("Num of images in test set {}".format(len(test_results.image_ids)))
exper_handler.test_results = test_results

INFO - Loading results from file /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02//stats/test_results_25imgs_mc10_fold0_ep150000.dll
INFO - Successfully loaded TestResult object.
Num of images in test set 25


In [196]:
image_range = None
u_type = "stddev"

analyze_slices(exper_handler, image_range=image_range, do_save=True, do_show=False, u_type=u_type,
              use_high_threshold=True, translate_img_range=True)

INFO - No image range specified using test_result object.image_ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
--------- Outlier slices -------------
[ 1 10]
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient016/slice_analysis_img_0_threshold0_1.pdf
--------- Outlier slices -------------
[1 2 4 7]
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient017/slice_analysis_img_1_threshold0_1.pdf
--------- Outlier slices -------------
[8]
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient018/slice_analysis_img_2_threshold0_1.pdf
--------- Outlier slices -------------
[11]
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient019/slice_analysis_img_3_threshol

In [5]:
image_range = [20,21,22,23,24]
errors_only = False

for image_num in image_range:
    exper_handler.test_results.visualize_uncertainty_histograms(image_num=image_num, width=20, height=60, 
                                                                info_type="uncertainty",
                                                                 do_save=True, slice_range=None,
                                                                 std_threshold=0., errors_only=errors_only, 
                                                                 do_show=False, use_bald=True,
                                                                 model_name = exper_handler.exper.model_name)
                                                                 # model_name="MC-dropout (0.1)") [0,1,2,3,4,5]
print("INFO - Ready!")

Rows/columns 14/4
p-values ttest/Mann-Withney-U 0.00E+00/0.00E+00 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient096/analysis_seg_err_slice1_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/4.67E-187 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient096/analysis_seg_err_slice2_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/2.36E-124 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient096/analysis_seg_err_slice3_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/4.14E-117 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient096/analysis_seg_err_slice4_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/1.19E-145 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_

p-values ttest/Mann-Withney-U 0.00E+00/2.46E-98 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice7_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/3.57E-84 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice8_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/3.10E-90 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice9_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/7.81E-87 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr2e02/figures/patient099/analysis_seg_err_slice10_mc10.pdf
p-values ttest/Mann-Withney-U 0.00E+00/3.68E-123 
INFO - Successfully saved fig /home/jorg/repository/dcnn_acdc/logs/20180418_15_02_05_dcnn_mcv1_150000E_lr

In [43]:
exper_handler = exper_hdl_mc005
exper_handler.get_outlier_stats(verbose=True)

Loaded outlier stats for 1 training epochs (property=outliers_per_epoch, dictionary (key=epoch) with tuple (6 elements) and property=test_results_per_epoch, dict (key=epoch) with tuple (4))


In [44]:

image_num = 5
# geth the patient id for this image_num
patient_id = test_set.img_file_names[image_num]
print("Patient ID {}".format(patient_id))
# get the "latest" (max epoch) outlier statistics, tuple with 6 dictionaries
outlier_stats = exper_handler.exper.outliers_per_epoch[exper_handler.exper.epoch_id]
image_outliers = ImageOutliers(patient_id, outlier_stats)
print(image_outliers.out_img_cls_es)
print(image_outliers.out_img_cls_ed)

Patient ID patient036
{7: [1, 2, 3]}
{6: [1], 7: [1, 2, 3]}
